In [1]:
import numpy as np
import pandas as pd
import codecs
import sqlite3
from collections import Counter

pd.set_option("display.max_columns", 80)

In [2]:
with codecs.open("../data/page_kind_blocked_0201.csv", 'r', 'utf-8', 'ignore') as f:
    kind = pd.read_csv(f,index_col=0)

In [ ]:
db_names = [
    "GAdata20170601-20170630.db",
    "GAdata20170701-20170731.db",
    "GAdata20170801-20170831.db",
    "GAdata20170901-20170930.db",
    "GAdata20171001-20171031.db",
    "GAdata20171101-20171130.db",
]
    
    
for db_name in db_names:
    con = sqlite3.connect('../data/{}'.format(db_name))
    cur = con.cursor()
    if db_name == "GAdata20170601-20170630.db":
        month = "201706"
    elif db_name == "GAdata20170701-20170731.db":
        month = "201707"
    elif db_name == "GAdata20170801-20170831.db":
        month = "201708"
    elif db_name == "GAdata20170901-20170930.db":
        month = "201709"
    elif db_name == "GAdata20171001-20171031.db":
        month = "201710"
    elif db_name == "GAdata20171101-20171130.db":
        month = "201711"
    elif db_name == "GAdata20171201-20171231.db":
        month = "201712"
    elif db_name == "GAdata20180101-20180131.db":
        month = "201801"
    elif db_name == "GAdata20180201-20180228.db":
        month = "201802"
    elif db_name == "GAdata20180301-20180331.db":
        month = "201803"
    elif db_name == "GAdata20180401-20180430.db":
        month = "201804"
    elif db_name == "GAdata20180501-20180531.db":
        month = "201805"
        
    
    for date in range(1,32):
        if date < 10:
            date = "0" + str(date)
        else:
            date =str(date)
        
        try:
            sql = "SELECT * FROM PV_{}{}".format(month,date)
            cur.execute(sql)

            visitId_list = []
            geoNetwork_country_list = []
            hits_page_pagePath_list = []
            device_deviceCategory_list = []
            geoNetwork_region_list = []
            trafficSource_keyword_list = []
            fullVisitorId_list = []
            trafficSource_source_list = []
            channelGrouping_list = []

            for row in cur:
                visitId_list.append(row[1])
                geoNetwork_country_list.append(row[4])
                hits_page_pagePath_list.append(row[6])
                device_deviceCategory_list.append(row[8])
                geoNetwork_region_list.append(row[10])
                trafficSource_keyword_list.append(row[11])
                fullVisitorId_list.append(row[12])
                trafficSource_source_list.append(row[14])
                channelGrouping_list.append(row[15])

            pv = pd.DataFrame()
            pv["visitId"] = visitId_list
            pv["geoNetwork_country"] = geoNetwork_country_list
            pv["url"] = hits_page_pagePath_list
            pv["device_deviceCategory"] = device_deviceCategory_list
            pv["geoNetwork_region"] = geoNetwork_region_list
            pv["trafficSource_keyword"] = trafficSource_keyword_list
            pv["fullVisitorId"] = fullVisitorId_list
            pv["trafficSource_source"] = trafficSource_source_list
            pv["channelGrouping"] = channelGrouping_list

            #セッションIDの追加
            pv["session_id"] = [str(pv["visitId"][i])+"_"+str(pv["fullVisitorId"][i])[:10] for i in range(len(pv))]

            #blockとマージ
            pv_block = pd.merge(pv, kind, on='url', how='left')
            pv_block["Block"] = pv_block["Block"].fillna("others")

            #Sページに到達したセッションIDを取得し、そのアクセスログだけのデータフレーム（pv_block_cv）を作成
            cv_session_id = pv_block[pv_block["Block"] == "CV"]["session_id"].unique()
            pv_block_cv = pv_block[pv_block["session_id"].isin(cv_session_id)]

            #cvページに行ったら、その後はもう予測はしない→それ以降のデータは削除する
            #その前に、CVページを消したらまずいので、先に正例・負例に分ける
            #正例のセッションID
            done_session_id = pv_block_cv[pv_block_cv["url"]=="showroom.lixil.co.jp/reserve/rsrv_done.php"]["session_id"].unique()

            #CVフラグ(0 or 1)はマージで挿入する
            cv_df = pd.DataFrame()
            cv_df["session_id"] = done_session_id
            cv_df["cv_flg"] = 1
            pv_block_cv_flg = pd.merge(pv_block_cv, cv_df, on='session_id', how='left')
            pv_block_cv_flg["cv_flg"] = pv_block_cv_flg["cv_flg"].fillna(0)

            feature_index=[]
            for session_id in pv_block_cv_flg["session_id"].unique():
                a = pv_block_cv_flg[pv_block_cv_flg["session_id"] == session_id]
                for i in range(len(a)):
                    if a.iloc[i]["Block"]!="CV":
                        feature_index.append(a.index[i])
                    else:
                        break

            pv_block_cv_flg_beforeCv = pv_block_cv_flg.loc[feature_index]



            ###############################変数作成###############################
            ###############Sブロック系###############

            #cvしたか
            cv_flg_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                cv_flg_list.append(a["cv_flg"].values[0])
            
            #閲覧ページ数
            num_of_pages_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                num_of_pages_list.append(len(a))
            
            #ブロック間移動回数
            num_of_move_block_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                num_move=0
                previous_page = ""
                for page in a["Block"].values:
                    if page != previous_page:
                        num_move+=1
                        previous_page = page

                num_of_move_block_list.append(num_move)

            #閲覧ブロックの種類の数
            num_of_kind_block_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                num_of_kind_block_list.append(len(a["Block"].unique()))

            #直前のブロック
            last_block_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                last_block_list.append(a.iloc[-1]["Block"])
                
            #ランディングブロック
            landing_block_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                landing_block_list.append(a["Block"].values[0])
            
            #検索の有無
            search_keyword_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                if (a["trafficSource_keyword"].isnull().values[0] == True) or (a["trafficSource_keyword"].values[0] == '(not provided)'):
                    search_keyword_list.append(0)
                else:
                    search_keyword_list.append(1)  

            #「ショールーム」と検索してるか？
            showroom_keyword_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                if (a["trafficSource_keyword"].isnull().values[0] == True) or (a["trafficSource_keyword"].values[0] == '(not provided)'):
                    showroom_keyword_list.append(0)
                else:
                    search_list = a["trafficSource_keyword"].values[0].split()
                    if "ショールーム" in search_list:
                        showroom_keyword_list.append(1) 
                    else:
                        showroom_keyword_list.append(0) 

            
            #検索した単語の数 
            num_keyword_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                if (a["trafficSource_keyword"].isnull().values[0] == True) or (a["trafficSource_keyword"].values[0] == '(not provided)'):
                    num_keyword_list.append(0)
                else:
                    search_list = a["trafficSource_keyword"].values[0].split()
                    num_keyword_list.append(len(search_list))
            
            
            feature_df = pd.DataFrame()
            feature_df["session_id"] = pv_block_cv_flg_beforeCv["session_id"].unique()
            feature_df["cv_flg"]=cv_flg_list
            feature_df["num_of_pages"]=num_of_pages_list
            feature_df["num_of_move_block"]=num_of_move_block_list
            feature_df["num_of_kind_block"]=num_of_kind_block_list
            feature_df["last_block"]=last_block_list
            feature_df["landing_block"]=landing_block_list
            feature_df["search_keyword"]=search_keyword_list
            feature_df["showroom_keyword"]=showroom_keyword_list
            feature_df["num_keyword"]=num_keyword_list
            
            
            ###############その他の変数###############
            
            #デバイスカテゴリ
            device_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                device_list.append(a["device_deviceCategory"].values[0])

            #地域情報
            region_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                region_list.append(a["geoNetwork_region"].values[0])

            #国情報
            country_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                country_list.append(a["geoNetwork_country"].values[0])

            #流入チャンネル
            channel_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                channel_list.append(a["channelGrouping"].values[0])

            #リファラソース
            referer_list=[]
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                referer_list.append(a["trafficSource_source"].values[0])
            
            feature_df["device_deviceCategory"] = device_list
            feature_df["geoNetwork_region"]=region_list
            feature_df["geoNetwork_country"]=country_list
            feature_df["channelGrouping"]=channel_list
            feature_df["trafficSource_source"]=referer_list
            
        
            block_dic = {}
            for block in kind["Block"].unique():
                block_dic["num_go_block_{}".format(block)] = [0]*len(feature_df)


            idx = 0
            for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
                a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
                for block,num in Counter(a["Block"]).items():
                    block_dic["num_go_block_{}".format(block)][idx] = num

                idx += 1

            for block in kind["Block"].unique():
                feature_df["num_go_block_{}".format(block)] = block_dic["num_go_block_{}".format(block)]


#             feature_df.to_csv("../data/feature/{0}/feature_df_{1}_{2}.csv".format(db_name,month,date))
            feature_df.to_csv("../data/feature/new_{0}/feature_df_{1}_{2}.csv".format(db_name,month,date))
            
        except sqlite3.OperationalError:
            pass

In [ ]:
block_dic = {}
for block in kind["Block"].unique():
    block_dic["num_go_block_{}".format(block)] = [0]*len(feature_df)


idx = 0
for session_id in pv_block_cv_flg_beforeCv["session_id"].unique():
    a = pv_block_cv_flg_beforeCv[pv_block_cv_flg_beforeCv["session_id"] == session_id]
    for block,num in Counter(a["Block"]).items():
        block_dic["num_go_block_{}".format(block)][idx] = num
        
    idx += 1
    
for block in kind["Block"].unique():
    feature_df["num_go_block_{}".format(block)] = block_dic["num_go_block_{}".format(block)]